# Assignment 4.1 - PCA

Please submit your solution of this notebook in the Whiteboard at the corresponding Assignment entry as .ipynb-file and as .pdf. <br><br>
Please do **NOT** rename the file!

#### State both names of your group members here:
[Jane and John Doe]

---

## Grading Info/Details - Assignment 4.1:

The assignment will be graded semi-automatically, which means that your code will be tested against a set of predefined test cases and qualitatively assessed by a human. This will speed up the grading process for us.

* For passing the test scripts: 
    - Please make sure to **NOT** alter predefined class or function names, as this would lead to failing of the test scripts.
    - Please do **NOT** rename the files before uploading to the Whiteboard!

* **(RESULT)** tags indicate checkpoints that will be specifically assessed by a human.

* You will pass the assignment if you pass the majority of test cases and we can at least confirm effort regarding the **(RESULT)**-tagged checkpoints per task.

---

## Task 4.1.1 - PCA from Scratch

Implement Principal Component Analysis (PCA) from scratch using only `NumPy`.
This assignment will help you understand the mathematical foundations of PCA.

* Implement the PCA given the class structure below. **(RESULT)**
* Test your implementation using small synthetic datasets described in the test functions below. **(RESULT)**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class PCA:
    """
    Principal Component Analysis implementation using only NumPy.
    """
    
    def __init__(self, n_components=2):
        """
        Initialize PCA.
        """
        self.n_components = n_components
        
    def fit(self):
        """
        Fit PCA on the training data X.
        """
        # TODO: Implement this function
        pass
    
    def transform(self):
        """
        Transform X into the principal component space.
        """
        # TODO: Implement this function
        pass
    
    def inverse_transform(self):
        """
        Transform data back to original space.
        """
        # TODO: Implement this function
        pass
    
    def fit_transform(self, X):
        """
        Fit PCA and transform X in one step.
        """
        self.fit(X)
        return self.transform(X)

In [ ]:
def test_basic_pca():
    """Test 1: Basic PCA on 2D data"""
    # TODO: Implement this function
    pass


def test_dimensionality_reduction():
    """Test 2: Reduce 5D data to 2D"""
    # TODO: Implement this function
    pass


def test_reconstruction():
    """Test 3: Inverse transform (reconstruction)"""
    # TODO: Implement this function
    pass


def test_variance_ordering():
    """Test 4: Components are ordered by variance"""
    print("Test 4: Verify components are ordered by explained variance")
    
    np.random.seed(42)
    X = np.random.randn(100, 5)
    
    pca = PCA(n_components=5)
    pca.fit(X)
    
    # Check that explained variances are in descending order
    variances = pca.explained_variance_
    is_sorted = np.all(variances[:-1] >= variances[1:])
    
    print(f"Explained variances: {variances}")
    print(f"Is sorted (descending): {is_sorted}")
    assert is_sorted, "Components not sorted by variance!"
    print("✓ Test 4 passed\n")


def test_centered_data():
    """Test 5: Verify data is properly centered"""
    print("Test 5: Verify data centering")
    
    np.random.seed(42)
    X = np.random.randn(100, 3) + 10  # Add offset
    
    pca = PCA(n_components=2)
    pca.fit(X)
    
    # Mean should be close to the original data mean
    print(f"Original data mean: {np.mean(X, axis=0)}")
    print(f"Stored mean: {pca.mean_}")
    print(f"Difference: {np.mean(np.abs(np.mean(X, axis=0) - pca.mean_)):.10f}")
    print("✓ Test 5 passed\n")


def run_all_tests():
    print("Running PCA test suite...\n")
    try:
        test_basic_pca()
        test_dimensionality_reduction()
        test_reconstruction()
        test_variance_ordering()
        test_centered_data()
        
        print("ALL TESTS PASSED!")
        
    except AssertionError as e:
        print(f"\n❌ Test failed: {e}")
    except Exception as e:
        print(f"\n❌ Unexpected error: {e}")

In [ ]:
# Run the test suite
run_all_tests()

## Task 4.1.2 - PCA on Real-World Data

* Apply your PCA implementation on the `California Housing Dataset`. **(RESULT)**
* Compare your results with those obtained from the scikit-learn PCA implementation: `sklearn.decomposition.PCA`. Are your results within numerical precision? **(RESULT)**


In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.decomposition import PCA as SklearnPCA

## Congratz, you made it! :)